In [18]:
import pandas as pd
trainfile='C:/Users/nwang/Desktop/nlp/code/q2_balance_339_5_train.csv'
testfile='C:/Users/nwang/Desktop/nlp/code/q2_balance_113_5_test.csv'

train_df = pd.read_csv(trainfile)
train_df.head(5)
test_df = pd.read_csv(testfile)
test_df.head(5)

,text,x,Recommend_score,Satisfaction_Value,len
0,Setting up the new account was easy Is it poss...,0,9.0,10.0,19.0
1,Better customer service,3,0.0,0.0,3.0
2,It was so easy and quick,1,10.0,10.0,6.0
3,0,2,9.0,8.0,0.0
4,Clearly state options available given all cons...,3,10.0,7.0,9.0


,text,x,Recommend_score,Satisfaction_Value,len
0,I think everything was just fine I was in and ...,1,10.0,10.0,14.0
1,None it was perfect Very easy,1,10.0,10.0,7.0
2,She was great,1,10.0,10.0,4.0
3,The service representative did all he she coul...,0,10.0,10.0,22.0
4,Everything was quick easy and personalized Thanks,1,10.0,10.0,8.0


In [13]:
def one_hot(Y):
    Y = np.eye(4)[Y.reshape(-1)]
    return Y.astype(int)

In [14]:
import numpy as np
test_df[['target1','target2','target3','target4']]=pd.DataFrame(one_hot(test_df.x.values))
train_df[['target1','target2','target3','target4']]=pd.DataFrame(one_hot(train_df.x.values))

IndexError: index 4 is out of bounds for axis 0 with size 4

In [17]:
train_df.head(5)
test_df.head(5)

,text,x,Recommend_score,Satisfaction_Value,len,target1,target2,target3,target4
0,Setting up the new account was easy Is it poss...,0,9.0,10.0,19.0,1,0,0,0
1,Better customer service,3,0.0,0.0,3.0,0,0,0,1
2,It was so easy and quick,1,10.0,10.0,6.0,0,1,0,0
3,0,2,9.0,8.0,0.0,0,0,1,0
4,Clearly state options available given all cons...,3,10.0,7.0,9.0,0,0,0,1


,text,x,Recommend_score,Satisfaction_Value,len,target1,target2,target3,target4
0,I think everything was just fine I was in and ...,1,10.0,10.0,14.0,0,1,0,0
1,None it was perfect Very easy,1,10.0,10.0,7.0,0,1,0,0
2,She was great,1,10.0,10.0,4.0,0,1,0,0
3,The service representative did all he she coul...,0,10.0,10.0,22.0,1,0,0,0
4,Everything was quick easy and personalized Thanks,1,10.0,10.0,8.0,0,1,0,0


In [8]:
# l1=test_df.target4.values.tolist()

def overlapping_percentage(a, b):
    return sum(1 for x,y in zip(a,b) if str(x) ==str(y) ) / float(len(a))
overlapping_percentage(l1,pred)

0.035398230088495575

In [14]:
train_df[['text','target1','target2','target3','target4']].to_csv("C:/Users/nwang/Desktop/nlp/bert/data/q2_balance_113_5/4classes/train.tsv", sep='\t', index=False, header=False)
test_df[['text','target1','target2','target3','target4']].to_csv("C:/Users/nwang/Desktop/nlp/bert/data/q2_balance_113_5/4classes/dev.tsv", sep='\t', index=False, header=False)

In [2]:
train_df_bert = pd.DataFrame({
    'id':range(len(train_df)),
    'label':train_df['x'],
    'alpha':['a']*train_df.shape[0],
    'text': train_df["text"].replace(r'\n', ' ', regex=True)
})

train_df_bert.head()

dev_df_bert = pd.DataFrame({
    'id':range(len(test_df)),
    'label':test_df['x'],
    'alpha':['a']*test_df.shape[0],
    'text': test_df["text"].replace(r'\n', ' ', regex=True)
})

dev_df_bert.head()

,id,label,alpha,text
0,0,0,a,Setting up the new account was easy Is it poss...
1,1,3,a,Better customer service
2,2,1,a,It was so easy and quick
3,3,2,a,0
4,4,3,a,Clearly state options available given all cons...


,id,label,alpha,text
0,0,1,a,I think everything was just fine I was in and ...
1,1,1,a,None it was perfect Very easy
2,2,1,a,She was great
3,3,0,a,The service representative did all he she coul...
4,4,1,a,Everything was quick easy and personalized Thanks


In [68]:
train_df_bert.to_csv('C:/Users/nwang/Desktop/nlp/code/q2_balance_339_5_train.tsv', sep='\t', index=False, header=False)
dev_df_bert.to_csv('C:/Users/nwang/Desktop/nlp/code/q2_balance_113_5_dev.tsv', sep='\t', index=False, header=False)

# Start here

In [1]:
from tool import *
import torch
import pickle
from torch.utils.data import (DataLoader, RandomSampler, SequentialSampler,
                              TensorDataset)
from torch.nn import CrossEntropyLoss, MSELoss

from tqdm import tqdm_notebook, trange
import os
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM, BertForSequenceClassification
from pytorch_pretrained_bert.optimization import BertAdam, WarmupLinearSchedule

from multiprocessing import Pool, cpu_count
import convert_examples_to_features

import logging
logging.basicConfig(level=logging.INFO)

In [2]:
device = torch.device("cpu")

# The input data dir. Should contain the .tsv files (or other data files) for the task.
DATA_DIR = "C:/Users/nwang/Desktop/nlp/bert/data/q2_balance_113_5/"

# Bert pre-trained model selected in the list: bert-base-uncased,
# bert-large-uncased, bert-base-cased, bert-large-cased, bert-base-multilingual-uncased,
# bert-base-multilingual-cased, bert-base-chinese.
BERT_MODEL = 'C:/Users/nwang/Desktop/nlp/bert/data/vocab/bert-base-cased.tar.gz'

# The name of the task to train.
TASK_NAME = 'maura'

# The output directory where the model predictions and checkpoints will be written.
OUTPUT_DIR = f'outputs/{TASK_NAME}/'
REPORTS_DIR = f'reports/{TASK_NAME}_evaluation_report/'
CACHE_DIR = 'cache/'

In [3]:
if os.path.exists(REPORTS_DIR) and os.listdir(REPORTS_DIR):
        REPORTS_DIR += f'/report_{len(os.listdir(REPORTS_DIR))}'
        os.makedirs(REPORTS_DIR)
if not os.path.exists(REPORTS_DIR):
    os.makedirs(REPORTS_DIR)
    REPORTS_DIR += f'/report_{len(os.listdir(REPORTS_DIR))}'
    os.makedirs(REPORTS_DIR)
    
if os.path.exists(OUTPUT_DIR) and os.listdir(OUTPUT_DIR):
        raise ValueError("Output directory ({}) already exists and is not empty.".format(OUTPUT_DIR))
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)
    

In [4]:
MAX_SEQ_LENGTH = 128

TRAIN_BATCH_SIZE = 24
EVAL_BATCH_SIZE = 8
LEARNING_RATE = 2e-1 #change from -5
NUM_TRAIN_EPOCHS = 1
RANDOM_SEED = 42
GRADIENT_ACCUMULATION_STEPS = 1
WARMUP_PROPORTION = 0.1
OUTPUT_MODE = 'classification'

CONFIG_NAME = "config.json"
WEIGHTS_NAME = "pytorch_model.bin"

processor = BinaryClassificationProcessor()
train_examples = processor.get_train_examples(DATA_DIR)
train_examples_len = len(train_examples)

In [5]:
label_list = processor.get_labels() # [0,1,2,3] for classification
num_labels = len(label_list)
label_list, num_labels

(['[1,0,0,0]', '[0,1,0,0]', '[0,0,1,0]', '[0,0,0,1]'], 4)

In [15]:
label_list = ["0", "1","2","3"] # [0,1,2,3] for classification
num_labels = len(label_list)

In [16]:
num_train_optimization_steps = int(
    train_examples_len / TRAIN_BATCH_SIZE / GRADIENT_ACCUMULATION_STEPS) * NUM_TRAIN_EPOCHS

In [17]:
tokenizer = BertTokenizer.from_pretrained('C:/Users/nwang/Desktop/nlp/bert/data/vocab/bert-base-uncased-vocab.txt', do_lower_case=False)

INFO:pytorch_pretrained_bert.tokenization:loading vocabulary file C:/Users/nwang/Desktop/nlp/bert/data/vocab/bert-base-uncased-vocab.txt


In [18]:
label_map ={label: i for i, label in enumerate(label_list)}
train_examples_for_processing = [(example, label_map, MAX_SEQ_LENGTH, tokenizer, OUTPUT_MODE) for example in train_examples]

In [19]:
label_map

{'0': 0, '1': 1, '2': 2, '3': 3}

In [20]:
process_count = cpu_count() - 1
if __name__ ==  '__main__':
    print(f'Preparing to convert {train_examples_len} examples..')
    print(f'Spawning {process_count} processes..')
    with Pool(process_count) as p:
        train_features = list(tqdm_notebook(p.imap(convert_examples_to_features.convert_example_to_feature, train_examples_for_processing), total=train_examples_len))

Preparing to convert 339 examples..
Spawning 63 processes..


In [21]:
model = BertForSequenceClassification.from_pretrained(BERT_MODEL, cache_dir=CACHE_DIR, num_labels=num_labels)

INFO:pytorch_pretrained_bert.modeling:loading archive file C:/Users/nwang/Desktop/nlp/bert/data/vocab/bert-base-cased.tar.gz
INFO:pytorch_pretrained_bert.modeling:extracting archive file C:/Users/nwang/Desktop/nlp/bert/data/vocab/bert-base-cased.tar.gz to temp dir C:\Users\nwang\AppData\Local\Temp\37\tmplkhgpcmk
INFO:pytorch_pretrained_bert.modeling:Model config {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "type_vocab_size": 2,
  "vocab_size": 28996
}

INFO:pytorch_pretrained_bert.modeling:Weights of BertForSequenceClassification not initialized from pretrained model: ['classifier.weight', 'classifier.bias']
INFO:pytorch_pretrained_bert.modeling:Weights from pretrained model not used in BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform

In [22]:
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): BertLayerNorm()
              (dropout): Dropout(p=0.1)
            )
          )
          (intermediat

In [23]:
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
    ]

optimizer = BertAdam(optimizer_grouped_parameters,
                     lr=LEARNING_RATE,
                     warmup=WARMUP_PROPORTION,
                     t_total=num_train_optimization_steps)
global_step = 0
nb_tr_steps = 0
tr_loss = 0
logger.info("***** Running training *****")
logger.info("  Num examples = %d", train_examples_len)
logger.info("  Batch size = %d", TRAIN_BATCH_SIZE)
logger.info("  Num steps = %d", num_train_optimization_steps)
all_input_ids = torch.tensor([f.input_ids for f in train_features], dtype=torch.long)
all_input_mask = torch.tensor([f.input_mask for f in train_features], dtype=torch.long)
all_segment_ids = torch.tensor([f.segment_ids for f in train_features], dtype=torch.long)

if OUTPUT_MODE == "classification":
    all_label_ids = torch.tensor([f.label_id for f in train_features], dtype=torch.long)
elif OUTPUT_MODE == "regression":
    all_label_ids = torch.tensor([f.label_id for f in train_features], dtype=torch.float)
    
train_data = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_label_ids)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=TRAIN_BATCH_SIZE)

INFO:root:***** Running training *****
INFO:root:  Num examples = 339
INFO:root:  Batch size = 24
INFO:root:  Num steps = 14


In [24]:
model.train()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): BertLayerNorm()
              (dropout): Dropout(p=0.1)
            )
          )
          (intermediat

In [25]:
for _ in trange(int(NUM_TRAIN_EPOCHS), desc="Epoch"):
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0
    for step, batch in enumerate(tqdm_notebook(train_dataloader, desc="Iteration")):
        batch = tuple(t.to(device) for t in batch)
        input_ids, input_mask, segment_ids, label_ids = batch

        logits = model(input_ids, segment_ids, input_mask, labels=None)
        print(logits)

        loss_fct = CrossEntropyLoss()
        loss = loss_fct(logits.view(-1,4), label_ids.view(-1))


        if GRADIENT_ACCUMULATION_STEPS > 1:
            loss = loss / GRADIENT_ACCUMULATION_STEPS

        loss.backward()
        print("\r%f" % loss, end='')
        
        tr_loss += loss.item()
        nb_tr_examples += input_ids.size(0)
        nb_tr_steps += 1
        if (step + 1) % GRADIENT_ACCUMULATION_STEPS == 0:
            optimizer.step()
            optimizer.zero_grad()
            global_step += 1

Epoch:   0%|                                                                                     | 0/1 [00:00<?, ?it/s]

tensor([[ 0.6973, -0.2416,  0.0074, -0.6925],
        [ 0.2546, -0.3022, -0.1536, -0.6192],
        [ 0.8883, -0.3594, -0.0798, -0.5555],
        [ 0.5534, -0.2793, -0.1337, -0.6628],
        [ 1.1199,  0.0778, -0.3339, -0.4041],
        [ 0.8687, -0.2536, -0.1180, -0.6922],
        [ 0.6139,  0.0118, -0.0976, -0.4370],
        [ 0.5623, -0.0319, -0.3060, -0.3283],
        [ 0.4207, -0.2418, -0.1202, -0.6381],
        [ 0.7094,  0.1793, -0.2021, -0.5408],
        [ 0.4756, -0.4571,  0.0561, -0.8851],
        [ 1.1201, -0.1803, -0.0893, -0.6103],
        [ 0.6431, -0.0909, -0.0298, -0.6759],
        [ 0.5077,  0.0648, -0.0875, -0.8575],
        [ 0.5169,  0.1469, -0.0173, -0.6367],
        [ 0.5719, -0.3595, -0.2040, -0.7115],
        [ 0.6605, -0.0609, -0.3724, -0.7517],
        [ 0.6595, -0.1448, -0.3058, -0.6509],
        [ 0.6265, -0.1631, -0.1910, -0.2797],
        [ 0.4842,  0.0767, -0.1127, -0.6355],
        [ 1.0119,  0.2032, -0.3949, -0.5040],
        [ 0.6428, -0.2065, -0.0942

RuntimeError: index out of range at ..\aten\src\TH/generic/THTensorEvenMoreMath.cpp:193

In [ ]:
output_mode = OUTPUT_MODE

# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained('bert-base-cased', do_lower_case=False)

processor = BinaryClassificationProcessor()
train_examples = processor.get_train_examples(DATA_DIR)
train_examples_len = len(train_examples)

label_list = processor.get_labels() # [0,1,2,3] for binary classification
num_labels = len(label_list)

label_map = {label: i for i, label in enumerate(label_list)}
train_examples_for_processing = [(example, label_map, MAX_SEQ_LENGTH, tokenizer, output_mode) for example in train_examples]

process_count = 1
if __name__ ==  '__main__':
    print(f'Preparing to convert {train_examples_len} examples..')
    print(f'Spawning {process_count} processes..')
    with Pool(process_count) as p:
        train_features = list(tqdm(p.imap(convert_examples_to_features.convert_example_to_feature, train_examples_for_processing), total=train_examples_len))

with open("train_features", 'wb') as f:
    pickle.dump(train_features, f)